- Fragmentation (from [here](https://dl.acm.org/citation.cfm?id=2893731))
- $\text{fragmentation}(x) = 1 - \frac{x \times\#(\text{free chuncks of size } x)}{\text{free space}}$

In [1]:
%matplotlib ipympl
import random

https://github.com/matplotlib/jupyter-matplotlib

In [2]:
def add_var(alloc_size=30, alloc_time=1, dealloc_time=10):
    return {"alloc_size":alloc_size, "alloc_time":alloc_time, "dealloc_time":dealloc_time}

In [402]:
class memory:
    def __init__(self, size):
        self.size = size # memory size in bytes
        self.free_mem = size
        self.memory = [ False ] * size
    
    def __str__(self):
        str1  = "Size: {} bytes\n".format(str(self.size))
        str1 += "Free: {} bytes\n".format(str(self.free_mem))
        str1 += str(list(int(i) for i in self.memory))
        return str1
                
    def writeAllocation(self, position, size):
        self.memory[position: position+size] = [True] * size
            
    def isThereSpace(self, position, size):
        if position+size > self.size:
            return 0
        return sum(self.memory[position: position+size]) == 0

In [403]:
class random_allocated_memory(memory):
    def allocate(self, size):
        allocated = False
        while not allocated:
            pick_place_to_allocate = random.randint(0,self.size-1-size)
            if self.isThereSpace(pick_place_to_allocate, size):
                self.writeAllocation(pick_place_to_allocate, size)
                self.free_mem -= size
                allocated = True

In [404]:
class sequential(memory):
    def allocate(self, size):
        self.writeAllocation(self.size-self.free_mem, size)
        self.free_mem -= size

In [405]:
class mpattern:
    'Allocation pattern'
    
    def __init__(self, size, memory_class=random_allocated_memory):
        self.allocation_list = [] # allocation_list is a list of byte sizes of objects to be allocated.
        self.memory = memory_class(size)
    
    def __str__(self):
        str1 = "Allocations:\n" + str(self.allocation_list)
        str1 += "\nMemory:\n" + str(self.memory)
        str1 += "\nFragmentation (of 4 bytes): {:.3f}".format(self.get_fragmentation())
        return str1
        
    def add_var(self, alloc_size=30):
        self.allocation_list.append(alloc_size)
    
    def create_pattern(self, sizes, numbers):
        if not isinstance(sizes, list):
            sizes = [sizes]
        if not isinstance(numbers, list):
            numbers = [numbers]
        assert len(sizes)==len(numbers), "Have to supply integers or lists of the same length. " + str(sizes) + " " + str(numbers)
        for s, n in zip(sizes,numbers):
            for i in range(n):
                self.allocation_list.append(s)
                self.memory.allocate(s)
        
    def get_number_free_chunks(self, size):
        num = 0
        i = 0
        while i < self.memory.size:
            if self.memory.isThereSpace(i,size):
                num += 1
                i += size
            else:
                i += 1
        return num
        
    def get_fragmentation(self, size_x=4):
        if size_x > self.memory.free_mem:
            return 1
        num_free_chunks_of_size_x = self.get_number_free_chunks(size_x)
        return 1-size_x*num_free_chunks_of_size_x/self.memory.free_mem

In [406]:
mem = mpattern(200, random_allocated_memory)

In [407]:
mem.add_var(1)

In [408]:
mem.create_pattern(3,40)

In [409]:
#mem.allocation_list

In [410]:
mem.memory.allocate(3)

In [411]:
print(mem)

Allocations:
[1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Memory:
Size: 200 bytes
Free: 77 bytes
[0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
Fragmentation (of 4 bytes): 0.481


In [412]:
mem.get_fragmentation(30)

1.0

In [413]:
size = 512
sizes = [2, 3, 4, 5, 6] # the order in which these are allocated influences the result (at least in the random allocator)
numbers = [10, 5, 10, 5, 3]
allocators = ["random_allocated_memory", "sequential"]

In [414]:
fragmentations = []
results = {}
for al in allocators:
    results[al] = mpattern(size, eval(al))
    results[al].create_pattern(sizes, numbers)
    print(results[al])
results

Allocations:
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6]
Memory:
Size: 512 bytes
Free: 394 bytes
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 

{'random_allocated_memory': <__main__.mpattern at 0x7ff9fe52d588>,
 'sequential': <__main__.mpattern at 0x7ff9fe52d4a8>}

In [460]:
def runs(size, sizes, numbers, allocators, nruns=1):
    fragmentations = {}
    results = {}
    for al in allocators:
        print("Allocator: {}".format(str(al)))
        fragmentations[al] = []
        for n in range(nruns):
            results[al + str(n)] = mpattern(size, eval(al))
            results[al + str(n)].create_pattern(sizes, numbers)
            print(" Run: {0:d}, Fragmentation: {1:.3f}".format(n, results[al + str(n)].get_fragmentation()))
            #print(str(results[al + str(n)]).replace("\n","\n  "))
            #print(" Fragmentation: {}" + str(results[al + str(n)].get_fragmentation()))
            fragmentations[al].append(results[al + str(n)].get_fragmentation())
    return fragmentations

In [468]:
runs_dict = runs(size, sizes, numbers, allocators, 3)
runs_dict

Allocator: random_allocated_memory
 Run: 0, Fragmentation: 0.137
 Run: 1, Fragmentation: 0.117
 Run: 2, Fragmentation: 0.096
Allocator: sequential
 Run: 0, Fragmentation: 0.005
 Run: 1, Fragmentation: 0.005
 Run: 2, Fragmentation: 0.005


{'random_allocated_memory': [0.13705583756345174,
  0.11675126903553301,
  0.09644670050761417],
 'sequential': [0.005076142131979711,
  0.005076142131979711,
  0.005076142131979711]}

In [504]:
import matplotlib.pyplot as plt
import numpy as np
def plot_runs(runs_dict):
    fig, ax = plt.subplots()
    labels = []
    fragmentation = []
    stdevs = []
    for  allocators, frag_list in runs_dict.items():
        labels.append(allocators)
        fragmentation.append(np.mean(frag_list))
        stdevs.append(np.std(frag_list))
    y_pos = np.arange(len(labels))
    ax.bar(y_pos, fragmentation, yerr=stdevs)#, align='center',
           #color='green', ecolor='black'
         # )
    ax.set_xticks(y_pos)
    ax.set_xticklabels(labels)
    ax.set_ylabel('Fragmentation')
    ax.set_title('Memz')

    plt.show()

In [505]:
plot_runs(runs_dict)

FigureCanvasNbAgg()

Some examples:

In [27]:
import matplotlib.pyplot as plt
plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt


plt.rcdefaults()
fig, ax = plt.subplots()

# Example data
people = ('Tom', 'Dick', 'Harry', 'Slim', 'Jim')
y_pos = np.arange(len(people))
performance = 3 + 10 * np.random.rand(len(people))
error = np.random.rand(len(people))

ax.barh(y_pos, performance, xerr=error, align='center',
        color='green', ecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(people)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Performance')
ax.set_title('How fast do you want to go today?')

plt.show()

FigureCanvasNbAgg()

In [28]:
import matplotlib.pyplot as plt
plt.figure()
x = [1,2,3]
y = [4,5,6]
plt.plot(x,y)
plt.show()

FigureCanvasNbAgg()